In [ ]:
# Register device request
import socket
import sys
import json
import platform
import requests
import psutil
import netifaces
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QLabel, QPushButton, QProgressBar, QWizard, QWizardPage, QLineEdit, QTextEdit, QMessageBox, QFormLayout
from PyQt5.QtCore import Qt

class Installer(QWizard):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Stormcloud Installer")
        self.setFixedSize(640, 480)

        self.addPage(WelcomePage())
        self.addPage(APIKeyPage())
        self.addPage(SystemInfoPage())
        self.addPage(InstallPage())
        self.addPage(FinishPage())

class WelcomePage(QWizardPage):
    def __init__(self):
        super().__init__()
        layout = QVBoxLayout()
        label = QLabel("Welcome to the Stormcloud Installer. Click 'Next' to begin.")
        layout.addWidget(label)
        self.setLayout(layout)

class APIKeyPage(QWizardPage):
    def __init__(self):
        super().__init__()
        layout = QVBoxLayout()
        label = QLabel("Enter your API key:")
        self.api_key_edit = QLineEdit()
        layout.addWidget(label)
        layout.addWidget(self.api_key_edit)
        self.setLayout(layout)

    def validatePage(self):
        api_key = self.api_key_edit.text()
        if self.validate_api_key(api_key):
            self.wizard().api_key = api_key
            return True
        else:
            QMessageBox.warning(self, "Invalid API Key", "The entered API key is invalid or could not be verified. Please try again.")
            return False

    def validate_api_key(self, api_key):
        # Replace this URL with your actual API URL for validation
        url = "https://www2.darkage.io:8443/api/validate-api-key"
        headers = {"Content-Type": "application/json"}
        data = {"api_key": api_key}        
        
        try:
            response = requests.post(url, headers=headers, json=data)
            if response.status_code == 200:
                QMessageBox.information(self, "API Key Validated", "The API key has been successfully validated!")
                return True
            else:
                return False
        except Exception as e:
            print(e)
            return False

class SystemInfoPage(QWizardPage):
    def __init__(self):
        super().__init__()
        self.setTitle("System Information")
        self.system_info = self.get_system_info()

        layout = QFormLayout()
        for key, value in self.system_info.items():
            layout.addRow(QLabel(key), self.createReadOnlyText(str(value)))
        self.setLayout(layout)

    def initializePage(self):
        self.system_info = self.get_system_info()
        self.system_info_text.setPlainText(json.dumps(self.system_info, indent=2))

    def get_system_info(self):
        system_info = {
            "hostname": socket.gethostname(),
            "ip_address": self.get_ipv4_address_associated_with_default_gateway(),
            "available_ram": str(round(psutil.virtual_memory().available / 1073741824, 1)) + " GB",
            "total_ram": str(round(psutil.virtual_memory().total / 1073741824, 1)) + " GB",
            "operating_system": platform.platform(),
            "device_name": "foo"
            # Add any other diagnostic information here
        }
        return system_info

    def get_ipv4_address_associated_with_default_gateway(self):
        default_gateway = netifaces.gateways()['default'][netifaces.AF_INET]
        default_gateway_interface = default_gateway[1]
        ipv4_addresses = [
            addr['addr']
            for addr in netifaces.ifaddresses(default_gateway_interface)[netifaces.AF_INET]
        ]
        return ipv4_addresses[0] if ipv4_addresses else None

    def validatePage(self):
        api_key = self.wizard().api_key
        self.system_info["api_key"] = api_key
        self.system_info["request_type"] = "register_new_device"
        self.system_info["device_status"] = 1
        self.system_info["device_type"] = "bar"
        result = self.register_new_device(self.system_info)
        if result:
            return True
        else:
            QMessageBox.warning(self, "Error", "Failed to register the new device. Please try again.")
            return False

    def register_new_device(self, data):
        url = "https://www2.darkage.io:8443/api/register-new-device"
        headers = {"Content-Type": "application/json"}

        try:
            response = requests.post(url, headers=headers, json=data)
            if response.status_code == 200:
                return True
            else:
                return False
        except:
            return False
    def createReadOnlyText(self, text):
        readOnlyText = QTextEdit()
        readOnlyText.setPlainText(text)
        readOnlyText.setReadOnly(True)
        readOnlyText.setFixedHeight(25)
        readOnlyText.setStyleSheet("""
            QTextEdit {
                background-color: #F0F0F0;
                color: #333;
                border: none;
            }
        """)
        return readOnlyText

class InstallPage(QWizardPage):
    # TODO: Write code to do stormcloud installer stuff
    
    def __init__(self):
        super().__init__()

        layout = QVBoxLayout()
        label = QLabel("Installing Stormcloud...")
        self.progress = QProgressBar()
        self.progress.setRange(0, 100)
        layout.addWidget(label)
        layout.addWidget(self.progress)
        self.setLayout(layout)

    def initializePage(self):
        self.progress.setValue(0)
        self.install()

    def install(self):
        # Replace this URL with the actual download URL of your application
        url = "https://example.com/my-cloud-backup.zip"
        response = requests.get(url, stream=True)

        total_size = int(response.headers.get('content-length', 0))
        chunk_size = 1024
        downloaded_size = 0

        with open("my-cloud-backup.zip", "wb") as f:
            for data in response.iter_content(chunk_size):
                downloaded_size += len(data)
                f.write(data)
                self.progress.setValue(int(downloaded_size * 100 / total_size))

        # Extract and perform the actual installation here
        # ...

class FinishPage(QWizardPage):
    def __init__(self):
        super().__init__()

        layout = QVBoxLayout()
        label = QLabel("Installation completed! Click 'Finish' to close the installer.")
        layout.addWidget(label)
        self.setLayout(layout)

if __name__ == "__main__":
    app = QApplication(sys.argv)

    window = Installer()
    window.show()

    sys.exit(app.exec_())

AttributeError: 'SystemInfoPage' object has no attribute 'system_info_text'